# **데이터 수집 : 코스피 개별 종목의 종목 코드로 재무정보 가져오기**
종목코드 -> 고유번호 -> 법인등록번호 -> 재무정보
1. '금융감독원_고유번호'를 통해 '종목 코드'로 '고유번호' 가져오기
    - [**금융감독원_고유번호**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018)
2. '금융감독원_공시정보_기업개황'에서 '고유번호'로 '법인등록번호' 가져오기
    - [**금융감독원_공시정보_기업개황**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002)
    - KOSPI200 기업 목록 활용
3. 가져온 '법인등록번호'으로 재무정보 가져오기
    - [**금융위원회_기업 재무정보**](https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043459)
---
- [DART](https://dart.fss.or.kr/main.do) : 금융감독원에서 운영하는 기업정보전자공시시스템
- FSC : 금융위원회

In [19]:
import json
import os
import sys
from glob import glob
from io import BytesIO
from zipfile import ZipFile
import time

import numpy as np
import pandas as pd
import requests
import xmltodict
from bs4 import BeautifulSoup

sys.path.append("../package")


import gitig_auth
import project_module as pmod
import project_variable as pvar
data = os.path.relpath(pvar.dir["data"])

# 종목코드 -> 고유번호
1. '금융감독원_고유번호'를 통해 '종목 코드'로 '고유번호' 가져오기
    - [**금융감독원_고유번호**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018)

In [20]:
# API로 데이터 가져와서 pandas.DataFrame으로 변환 => df_cc (cc:corp_code)
auth_key = gitig_auth.auth_key_dict["dart"]
url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={auth_key}"
response = requests.get(url)

with ZipFile(BytesIO(response.content)) as df_cc:
    df_cc = df_cc.read("CORPCODE.xml")
    df_cc = pd.read_xml(df_cc)

df_cc.head()

,corp_code,corp_name,stock_code,modify_date
0,434003,다코,NaN,20170630
1,434456,일산약품,NaN,20170630
2,430964,굿앤엘에스,NaN,20170630
3,432403,한라판지,NaN,20170630
4,388953,크레디피아제이십오차유동화전문회사,NaN,20170630


In [27]:
df_cc = df_cc.dropna(subset=["stock_code"])
df_cc["corp_code"] = df_cc["corp_code"].astype(str).apply(lambda x: x.zfill(8))
df_cc["stock_code"] = df_cc["stock_code"].astype(str).apply(lambda x: x.zfill(6))
df_cc["jurir_no"] = None
df_cc = df_cc[["stock_code", "corp_code", "jurir_no", "corp_name"]]
df_cc.head()

,stock_code,corp_code,jurir_no,corp_name
2009,36720.0,00260985,None,한빛네트
2021,40130.0,00264529,None,엔플렉스
2022,55000.0,00358545,None,동서정보기술
2784,32600.0,00231567,None,애드모바일
3889,37600.0,00247939,None,씨모스


# 고유번호 -> 법인등록번호
2. '금융감독원_공시정보_기업개황'에서 '고유번호'로 '법인등록번호' 가져오기
    - [**금융감독원_공시정보_기업개황**](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002)
    - KOSPI200 기업 목록 활용

In [24]:
# 종목코드 -> 고유번호
def stockCode_to_corpCode(df, stock_code):
    return df[df["stock_code"] == stock_code]["corp_code"].values


# 고유번호 -> 법인등록번호
def corpCode_to_jurirNo(
    corp_code,
    auth_key="76dd702fe016b819f3e728f9a63b3b13eadaaeb4",
    url="https://opendart.fss.or.kr/api/company.json",
):
    params = {"crtfc_key": auth_key, "corp_code": corp_code}

    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()["jurir_no"]
        df.loc[
            df["corp_code"] == stockCode_to_corpCode(df, stock_code), "jurir_no"
        ] = corpCode_to_jurirNo(stockCode_to_corpCode(df, stock_code))

    else:
        print(response.status_code)
    time.sleep(0.3)

In [9]:
df_ksp = pd.read_csv(r"data\KOSPI200.csv")
df_ksp["종목코드"] = df_ksp["종목코드"].astype(str).apply(lambda x: x.zfill(6))
df_ksp["종목코드"].to_list()

tmp = df_ksp["종목코드"].to_list()
for stock_code in tmp:
    df_cc.loc[df_cc['corp_code'] == stockCode_to_corpCode(df_cc, stock_code), 'jurir_no'] = corpCode_to_jurirNo(stockCode_to_corpCode(df_cc, stock_code))
    
    
    


df_cc["jurir_no"] = df_cc["jurir_no"].astype(str).apply(lambda x: x.zfill(13))
spac = df_cc[df_cc['jurir_no'].str.contains('0000000000000')].index
df_cc.drop(spac, inplace=True)
df_cc

KeyError: 'jurir_no'

In [25]:
# KOSPI200 종목코드 리스트 가져오기
df_ksp = pd.read_csv(r"data\KOSPI200.csv")
df_ksp["종목코드"] = df_ksp["종목코드"].astype(str).apply(lambda x: x.zfill(6))


# 가져온 리스트로 법인등록번호 가져오기
temp = pd.unique(df_ksp["종목코드"]).tolist()
for stock_code in temp:
    df_cc.loc[df_cc["corp_code"] == stockCode_to_corpCode(df_cc, stock_code), "jurir_no"] = corpCode_to_jurirNo(stockCode_to_corpCode(df_cc, stock_code))

KeyError: 'jurir_no'